#Team Members:
1. Sachin Balakrishnan
2. Mohini Agrawal
3. Anshika Ahuja
4. Pooja Dawada

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:

!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver is already the newest version (78.0.3904.97-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


# Task A: Scraper

In [0]:
import pandas as pd

driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [0]:
#Top 250 beers
links_dict = {}
df_base=pd.DataFrame(columns=['beer_name','rating','link'])
driver.get('https://www.beeradvocate.com/beer/top-rated/')
for i in range(0,250):
  links_xpath = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr['+str(i+2)+']/td[2]/a')[0]
  link =links_xpath.get_attribute('href')
  name_xpath = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr['+str(i+2)+']/td[2]/a/b')[0]
  name = name_xpath.text
  rating_xpath = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr['+str(i+2)+']/td[4]/b')[0]
  rating = rating_xpath.text
  links_dict.update({name:link})
  df_base.loc[len(df_base)]=[name, rating, link]

In [0]:
df_base

,beer_name,rating,link
0,Kentucky Brunch Brand Stout,4.9,https://www.beeradvocate.com/beer/profile/2322...
1,Marshmallow Handjee,4.81,https://www.beeradvocate.com/beer/profile/26/4...
2,Hunahpu's Imperial Stout - Double Barrel Aged,4.79,https://www.beeradvocate.com/beer/profile/1798...
3,Barrel-Aged Abraxas,4.79,https://www.beeradvocate.com/beer/profile/2588...
4,Heady Topper,4.76,https://www.beeradvocate.com/beer/profile/4631...
...,...,...,...
245,Enjoy By IPA,4.44,https://www.beeradvocate.com/beer/profile/147/...
246,Scaled Up,4.46,https://www.beeradvocate.com/beer/profile/3065...
247,Dragonsaddle,4.5,https://www.beeradvocate.com/beer/profile/3017...
248,Flora - Blueberry,4.52,https://www.beeradvocate.com/beer/profile/2251...


In [0]:
links_dict

{'3rd Anniversary Imperial IPA': 'https://www.beeradvocate.com/beer/profile/33510/260703/',
 '4th Anniversary': 'https://www.beeradvocate.com/beer/profile/33510/325706/',
 'A Deal With The Devil': 'https://www.beeradvocate.com/beer/profile/24903/89499/',
 'Aaron': 'https://www.beeradvocate.com/beer/profile/22511/126517/',
 'Abner': 'https://www.beeradvocate.com/beer/profile/22511/58299/',
 'Abrasive Ale': 'https://www.beeradvocate.com/beer/profile/13014/46849/',
 'Abraxas': 'https://www.beeradvocate.com/beer/profile/25888/76393/',
 'Abricot Du Fermier': 'https://www.beeradvocate.com/beer/profile/32319/137889/',
 'Abt 12': 'https://www.beeradvocate.com/beer/profile/259/1708/',
 'Adam From The Wood': 'https://www.beeradvocate.com/beer/profile/173/20767/',
 'All That Is And All That Ever Will Be': 'https://www.beeradvocate.com/beer/profile/28743/249986/',
 'Alter Ego': 'https://www.beeradvocate.com/beer/profile/28743/176650/',
 'Anabasis': 'https://www.beeradvocate.com/beer/profile/32319/

In [0]:
#Reviews of the top 250 rated beers
import re
df=pd.DataFrame(columns=['beer_name','rating','review'])
for i in links_dict:
  driver.get(links_dict[i])
  for j in range(0,25):
    try:
      rating_xpath = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+str(j+1)+']/div[2]/span[1]')[0]
      rating = rating_xpath.text
      review_xpath = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+str(j+1)+']/div[2]')[0]
      review = review_xpath.text
      review = re.split('overall: \d\.?\d?\d?',review)[1]
      review = re.split("(\d+,?\d?\d?\d? characters)", review)[0]
      df.loc[len(df)]=[i, rating, review]
    except:
      break


In [0]:
df.head()

,beer_name,rating,review
0,Kentucky Brunch Brand Stout,4.8,"\n\n2016 Silver Wax. Aroma has whiskey, maple,..."
1,Kentucky Brunch Brand Stout,4.74,\n\nThe beer pours Pitch Black with a frothy t...
2,Kentucky Brunch Brand Stout,4.68,\n\nProbably the smoothest beer I have ever ha...
3,Kentucky Brunch Brand Stout,5,"\n\nDark black, very thick, a little bit of ta..."
4,Kentucky Brunch Brand Stout,4.97,\n\nPoured black as ink with thin ruby edges a...


In [0]:
df_base.to_csv('drive/My Drive/Text/Text 2/data_base.csv')
df.to_csv('drive/My Drive/Text/Text 2/data.csv')

#Task B: Attribute Selection

Removing stop words

In [0]:
 df.groupby('beer_name')['review'].count()

beer_name
3rd Anniversary Imperial IPA    25
4th Anniversary                 22
A Deal With The Devil           25
Aaron                           25
Abner                           25
                                ..
Westly                          25
Wide Awake It's Morning         25
Zenne Y Frontera                25
Zombie Dust                     25
§ucaba                          25
Name: review, Length: 250, dtype: int64

In [0]:
!pip install spacy

In [0]:
# Import the Data Frame
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('all')
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set( stopwords.words('english'))

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy 
import numpy as np
import pandas as pd
df_review = pd.read_csv('drive/My Drive/Text/Text 2/data.csv')
df_review = df_review.drop(['Unnamed: 0'],axis=1)


# Function to remove stopwords and replacing model names with brand names
# This function will be applied to all comments in our data frame

def filterReview(review):
  filtered_sentence = []
  tokens = word_tokenize(review)
  word_str = ""
  for w in tokens:
    if w.lower() not in stop_words: 
        word_str = word_str + " " + w
  return word_str

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
# New column to hold the comments after removing stopwords
df_review['Filtered review'] = df_review['review'].map(filterReview)

In [0]:
df_review.head()

,beer_name,rating,review,Filtered review
0,Kentucky Brunch Brand Stout,4.80,"\n\n2016 Silver Wax. Aroma has whiskey, maple,...","2016 Silver Wax . Aroma whiskey , maple , tof..."
1,Kentucky Brunch Brand Stout,4.74,\n\nThe beer pours Pitch Black with a frothy t...,beer pours Pitch Black frothy tan head . bott...
2,Kentucky Brunch Brand Stout,4.68,\n\nProbably the smoothest beer I have ever ha...,Probably smoothest beer ever . Smelled better...
3,Kentucky Brunch Brand Stout,5.00,"\n\nDark black, very thick, a little bit of ta...","Dark black , thick , little bit tan head . Sm..."
4,Kentucky Brunch Brand Stout,4.97,\n\nPoured black as ink with thin ruby edges a...,Poured black ink thin ruby edges 58 degrees ;...


In [0]:
# Getting the word counts from the user comments
from collections import Counter
s=df_review['Filtered review'].apply(word_tokenize)
text = s.apply(pd.Series).stack().reset_index(drop=True)
word_counts = Counter(text)
common_words = word_counts.most_common()
common_words

[('.', 45848),
 (',', 41780),
 ('beer', 4866),
 (':', 4479),
 ('head', 3542),
 ('-', 2553),
 ("'s", 2550),
 ('chocolate', 2540),
 ('dark', 2328),
 ('like', 2235),
 ('sweet', 2036),
 ('!', 1975),
 ('coffee', 1929),
 ('notes', 1905),
 ('one', 1895),
 ('bourbon', 1854),
 ('vanilla', 1807),
 ('nice', 1767),
 ('finish', 1747),
 ('well', 1685),
 ('good', 1636),
 ('nose', 1635),
 ('carbonation', 1615),
 ('light', 1613),
 ('fruit', 1580),
 ('bit', 1564),
 ('taste', 1562),
 ('orange', 1557),
 ('little', 1474),
 ('’', 1466),
 ('body', 1465),
 ('white', 1447),
 ('bottle', 1441),
 ('Taste', 1414),
 ('flavor', 1375),
 (')', 1365),
 (';', 1352),
 ('lacing', 1348),
 ('(', 1347),
 ('citrus', 1345),
 ('barrel', 1315),
 ('Pours', 1291),
 ('glass', 1290),
 ("n't", 1280),
 ('flavors', 1275),
 ('oak', 1273),
 ('black', 1256),
 ('smooth', 1255),
 ('really', 1180),
 ('...', 1160),
 ('color', 1149),
 ('brown', 1146),
 ('Overall', 1116),
 ('thick', 1114),
 ('malt', 1094),
 ('aroma', 1079),
 ('great', 1073),
 (


#Attribute
Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops

Complex: Multidimensional; many flavors and sensations on the palate

Crisp: Highly carbonated; effervescent

In [0]:
!pip install en_core_web_sm

#Task C: Similarity Score

In [0]:

import spacy 
  
nlp = spacy.load("en_core_web_sm")

In [0]:
import warnings; warnings.simplefilter('ignore')

In [0]:

doc2 = nlp("Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops. Complex: Multidimensional; many flavors and sensations on the palate. Crisp: Highly carbonated; effervescent")
def find_similarity(review):
  review_nlp = nlp(review)
  sim_score = review_nlp.similarity(doc2)
  return sim_score
df_review['sim_score'] = df_review['Filtered review'].apply(find_similarity)


#Task D:  Top 300 Sentiment Analysis

In [0]:
#df_review
Top_300 = df_review.sort_values('sim_score', ascending=False).head(300).reset_index(drop=True)
Top_300.to_csv('drive/My Drive/Text/Text 2/Top_300.csv')

# Sentiment

In [0]:
Top = pd.read_csv('drive/My Drive/Text/Text 2/Top_300.csv')
Top = Top.drop(['Unnamed: 0'],axis=1)

In [0]:
Top.head()

,beer_name,rating,review,Filtered review,sim_score
0,Heavy Mettle,4.74,\n\nA: Hazy gold/yellow with rich white head a...,: Hazy gold/yellow rich white head sticky lac...,0.879106
1,Abrasive Ale,4.77,\n\nPoured from a 16 oz can into a snifter. Ca...,Poured 16 oz snifter . Canned 10/26/16 . : Po...,0.872659
2,Doubleganger,4.52,\n\nServing: Can\nCanned: Apr-2-2019 (Consumed...,Serving : Canned : Apr-2-2019 ( Consumed Apr-...,0.867791
3,Blåbær Lambik,4.65,"\n\nVintage 2015\n\nA: Pours a dark purple, wi...","Vintage 2015 : Pours dark purple , ruby reddi...",0.866866
4,Congress Street IPA,3.84,\n\nCanned 10/27/16; drank 11/5/16 @ the Yarch...,Canned 10/27/16 ; drank 11/5/16 @ Yarchives ....,0.866352


In [0]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def get_sentiment(review, **kwargs):
 sentiment_score = analyser.polarity_scores(review)
 positive_meter = round((sentiment_score['pos'] * 100), 2)
 negative_meter = round((sentiment_score['neg'] * 100), 2)
 return positive_meter if kwargs['k'] == 'positive' else negative_meter

In [0]:
Top['positive'] = Top['Filtered review'].apply(get_sentiment, k='positive')
Top['negative'] = Top['Filtered review'].apply(get_sentiment, k='negative')


#Task E: Top 3 with similarity scores

In [0]:
aggregate = Top.groupby('beer_name')['sim_score','positive','negative'].mean()
aggregate.sort_values(by='positive', ascending=False).head(3)

,sim_score,positive,negative
beer_name,,,
Montmorency Vs Balaton,0.835020,48.7,0.0
Scaled Up,0.841315,47.6,0.0
Aurelian Lure,0.844844,42.2,0.0


#Task F: Top 3 without similarity

In [0]:
#Just taking top rated from toprated page
wo_sim_top = df_review[df_review['beer_name'].isin(df_base.head(3)['beer_name'])]
wo_sim_top['positive'] = wo_sim_top['Filtered review'].apply(get_sentiment, k='positive')
wo_sim_top['negative'] = wo_sim_top['Filtered review'].apply(get_sentiment, k='negative')
aggregate = wo_sim_top.groupby('beer_name')['sim_score','positive','negative'].mean()
aggregate

,sim_score,positive,negative
beer_name,,,
Hunahpu's Imperial Stout - Double Barrel Aged,0.749020,23.464,4.560
Kentucky Brunch Brand Stout,0.757998,28.468,4.544
Marshmallow Handjee,0.771086,34.268,2.296


#Insight


### Beer Attributes considered - Hoppy, Complex and Crisp

#### Our Recommendation: 
1. Montmorency Vs Balaton
2. Scaled way up
3. Aurelian Lure

As we can see in the dataframe above, our recommendations filter out the positive sentiments quite well. All are having a pretty high similarity scores in excess of 0.8 and we couldn’t find much negative sentiments either associated with the attributes. 

After manually glancing through the user reviews for these, we could find that people love the Cherry flavor and the hit of Citrus and raspberry in 'Montmorency Vs Balaton' which backs up the 'hoppy' preference of our user. Same is the case with our second recommendation - 'Scaled Way Up', which is highly regarded for its malty and citrus flavor of oranges, peach, tangerines and grapefruit. Users who love the sourness of citrus rated the 'Aurelian Lure' quite high. This suggests all of our recommendations have a rich mix of fruity flavours and do justice to the user preference of 'complex' and ‘hoppy’ beers. 

They have high positive reviews for their med-high carbonation as well. Scaled way up has been rated for its sharp carbonation and silky feel on the palate. People love the fizziness of ‘Montmorency Vs Balaton’ and the balanced carbonation of ‘Aurelian Lure’. 

#### Top Rates Beers from the dataset:
1. Hunahpu's Imperial Stout - Double Barrel Aged
2. Kentucky Brunch Brand Stout
3. Marshmallow Handjee

From the user reviews of these top-rated beers, we could understand that none of these matches well with the hoppy, fruity and citrus flavours that our user prefers. ‘Marshmallow Handjee’ is known for its vanilla and chocolate flavours. Users rate ‘Hunahpu's Imperial Stout’ for its hit of cinnamon and pepper combined with the feel of spiced rum. And it seems the 'Kentucky Brunch Brand Stout' is high on the smokey coffee and vanilla flavors and users love its creaminess.

We could infer that these 3 beers are quite good when considered on the specific tastes they offer, but lack variety and a mix of fruity flavours that our user loves.  


In addition, we found that all three are mild-med carbonated beers. ‘Marshmallow Handjee’ has a mixed review where one user suggested that it is low on carbonation while another rated it sharp. Users love the smoothness of ‘Kentucky Brunch Brand Stout’, however we could find a few complaining about the lack of carbonation. ‘Hunahpu's Imperial Stout’ is syrupy and users unanimously expressed their mild-med carbonation preference there.
